**Tic-Tac-Toe Game in Python**

# **Setting Up the Game Board**

create the basic game board structure:

In [2]:
class TicTacToe:
    def __init__(self):
        # Initialize a 3x3 board with empty spaces
        self.board = [' ' for _ in range(9)]  # Single list represents the board
        self.current_winner = None  # Tracks if we have a winner

    def print_board(self):
        # Display the current board state
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # Show which numbers correspond to which positions
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        # Returns list of indices of empty squares
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        # Are there any empty squares left?
        return ' ' in self.board

    def num_empty_squares(self):
        # Count of empty squares
        return len(self.available_moves())

    def make_move(self, square, letter):
        # Make a move if valid, return True if valid move
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # Check if the current move caused a win

        # Check row
        row_ind = square // 3
        row = self.board[row_ind*3 : (row_ind + 1)*3]
        if all([spot == letter for spot in row]):
            return True

        # Check column
        col_ind = square % 3
        column = [self.board[col_ind + i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        # Check diagonals (only if the move is on a diagonal space)
        if square % 2 == 0:  # Diagonal spaces are 0, 2, 4, 6, 8
            diagonal1 = [self.board[i] for i in [0, 4, 8]]  # Top-left to bottom-right
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]  # Top-right to bottom-left
            if all([spot == letter for spot in diagonal2]):
                return True
        return False

# **Creating Player Types**

Now let's implement the different player types

In [3]:
import random
import time
import math

class HumanPlayer:
    def __init__(self, letter):
        # Letter is X or O
        self.letter = letter

    def get_move(self, game):
        # Get valid human input
        valid_square = False
        val = None
        while not valid_square:
            square = input(f'{self.letter}\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        # Random valid move
        square = random.choice(game.available_moves())
        return square

class SmartComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            # First move - choose a random corner for better strategy
            square = random.choice([0, 2, 6, 8])
        else:
            # Use minimax algorithm to find best move
            square = self.minimax(game, self.letter)['position']
        return square

    def minimax(self, state, player):
        max_player = self.letter  # The AI player
        other_player = 'O' if player == 'X' else 'X'

        # Base cases - check if game is over
        if state.current_winner == other_player:
            return {'position': None,
                    'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player
                    else -1 * (state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        # Initialize best move
        if player == max_player:
            best = {'position': None, 'score': -math.inf}  # Maximize
        else:
            best = {'position': None, 'score': math.inf}  # Minimize

        for possible_move in state.available_moves():
            # 1. Try the move
            state.make_move(possible_move, player)
            # 2. Simulate game after that move
            sim_score = self.minimax(state, other_player)
            # 3. Undo the move
            state.board[possible_move] = ' '
            state.current_winner = None
            # 4. Update best move
            sim_score['position'] = possible_move

            if player == max_player:  # Maximizing player
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:  # Minimizing player
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best

# **Game Play Function**

Here's the function that manages the game flow:

In [4]:
def play(game, x_player, o_player, print_game=True):
    # Main game function that handles turns and game state

    if print_game:
        game.print_board_nums()

    letter = 'X'  # Starting player

    while game.empty_squares():
        # Get the move from the appropriate player
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)

        # Make the move and check for winner
        if game.make_move(square, letter):
            if print_game:
                print(f'{letter} makes a move to square {square}')
                game.print_board()
                print('')  # Empty line

            if game.current_winner:
                if print_game:
                    print(f'{letter} wins!')
                return letter

            # Switch players
            letter = 'O' if letter == 'X' else 'X'

        # Small delay for computer moves
        time.sleep(0.8)

    # If we exit the loop and no winner, it's a tie
    if print_game:
        print('It\'s a tie!')
    return None

# **Player Selection and Main Game Loop**

Finally, let's add the player selection and main game loop:

In [ ]:
def select_player_type(letter):
    # Let user choose player type for X or O
    while True:
        choice = input(f"Select player type for {letter}:\n1. Human\n2. Random Computer\n3. Smart Computer\nEnter choice (1-3): ")
        if choice in ['1', '2', '3']:
            if choice == '1':
                return HumanPlayer(letter)
            elif choice == '2':
                return RandomComputerPlayer(letter)
            else:
                return SmartComputerPlayer(letter)
        print("Invalid choice. Please enter 1, 2, or 3.")

def main():
    print("Welcome to Tic Tac Toe!")
    print("Here are the positions on the board:")

    while True:
        # Select player types
        print("\nSelect players for this game:")
        x_player = select_player_type('X')
        o_player = select_player_type('O')

        # Start the game
        t = TicTacToe()
        play(t, x_player, o_player, print_game=True)

        # Ask to play again
        play_again = input("Do you want to play again? (y/n): ").lower()
        if play_again != 'y':
            print("Thanks for playing!")
            break

# Start the game when running in Colab
if __name__ == '__main__':
    main()

Welcome to Tic Tac Toe!
Here are the positions on the board:

Select players for this game:
Select player type for X:
1. Human
2. Random Computer
3. Smart Computer
Enter choice (1-3): 1
Select player type for O:
1. Human
2. Random Computer
3. Smart Computer
Enter choice (1-3): 2
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X's turn. Input move (0-8): 4
X makes a move to square 4
|   |   |   |
|   | X |   |
|   |   |   |

O makes a move to square 3
|   |   |   |
| O | X |   |
|   |   |   |

X's turn. Input move (0-8): 5
X makes a move to square 5
|   |   |   |
| O | X | X |
|   |   |   |

O makes a move to square 6
|   |   |   |
| O | X | X |
| O |   |   |

X's turn. Input move (0-8): 0
X makes a move to square 0
| X |   |   |
| O | X | X |
| O |   |   |

O makes a move to square 7
| X |   |   |
| O | X | X |
| O | O |   |

X's turn. Input move (0-8): 8
X makes a move to square 8
| X |   |   |
| O | X | X |
| O | O | X |

X wins!
